In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
import plotly.graph_objects as go
import pandas as pd
from sklearn.linear_model import LassoCV

import tensorflow_probability as tfp 
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
from tensorflow_probability import distributions as tfd
from datetime import datetime, timedelta
import tensorflow.compat.v2.keras as keras
from datetime import datetime
from sklearn import linear_model
from scipy.stats import norm, johnsonsu


In [ ]:
data= pd.read_csv("Final_dataset.csv")
data['DateTime'] = pd.to_datetime(data['DateTime'])
data = data.drop(columns= ['is_holiday', 'temp_actual','wspd_actual', 'temp_24hour_MA', 'temp_48hour_MA', 'temp_72hour_MA','Name',])
data.set_index('DateTime', inplace=True)
#data = data.loc['2023-12-01 09:00:00':] # 2023-04-01 to 2024-03-31
data

In [ ]:
# weather_forecast = pd.read_csv("weatherforecast.csv")
# weather_forecast = weather_forecast[(weather_forecast['horizon'] >= 1) & (weather_forecast['horizon'] <= 168)]
# weather_forecast = weather_forecast.drop(columns= ['forecast', 'wspd_forecast'])
# weather_forecast['DateTime'] = pd.to_datetime(weather_forecast['DateTime'])
# weather_forecast.set_index('DateTime', inplace=True)
# weather_forecast.index = weather_forecast.index.tz_convert(None)
# weather_forecast.reset_index(inplace=True)
# data.reset_index(inplace=True)
# data_expanded = data.loc[data.index.repeat(168)].reset_index(drop=True)
# data_expanded['horizon'] = list(range(1, 169)) * len(data)
# combined_data = pd.merge(data_expanded, weather_forecast, on=['DateTime', 'horizon'], how='left')
# combined_data.set_index('DateTime', inplace=True)



## Neural networks

In [ ]:
#data.set_index('DateTime', inplace=True)
#data.index = pd.to_datetime(data.index) 

In [ ]:
distribution = 'JSU' # distribution for forecast
paramcount = {'Normal': 2,
              'JSU': 4
              }
INP_SIZE =  123
S=24
activations = ['sigmoid', 'relu', 'elu', 'tanh', 'softplus', 'softmax']

In [ ]:
def runoneday(inp):
    params, dayno = inp
    
    # Define the start date for the forecast based on dayno
    start_date = pd.Timestamp('2023-12-01 09:00:00') + pd.Timedelta(days=dayno)
    
   
    df = data.loc[start_date:] 
    

    # Compute the size of the dataset
    num_days = len(df) // 24
    

    
    # Initialize Y matrix for target values
    Y = np.zeros((num_days - 7, 168))  
    for d in range(num_days - 7):
        start_idx = d * 24
        end_idx = (d + 7) * 24
        Y[d, :] = df.loc[df.index[start_idx:end_idx], 'Load_DA'].to_numpy()
        

    # Initialize X matrix for input features
    X = np.zeros((num_days, INP_SIZE))  # Adjust X size for the full dataset
    for d in range(7, num_days):
        
        X[d, :24] = df.loc[df.index[(d-1)*S:d*S], 'Load_DA'].to_numpy()  # D-1 load
        X[d, 24:48] = df.loc[df.index[(d-2)*S:(d-1)*S], 'Load_DA'].to_numpy()  # D-2 load
        X[d, 48:72] = df.loc[df.index[(d-7)*S:(d-6)*S], 'Load_DA'].to_numpy()  # D-7 load
        
        X[d, 72:96] = df.loc[df.index[d*S:(d+1)*S], 'is_flex_holiday'].to_numpy()  # is_flex_holiday
        X[d, 96:120] = df.loc[df.index[d*S:(d+1)*S], 'is_fixed_holiday'].to_numpy()  # is_fixed_holiday
        X[d, 120] = df.loc[df.index[d*S:(d+1)*S:S], 'is_regional_holiday'].to_numpy()  # is_regional_holiday
        X[d, 121] = df.loc[df.index[d*S:(d+1)*S:S], 'is_xmas'].to_numpy()  # is_xmas
        X[d, 122] = df.index[d * 24].weekday()  # weekday 
        

    
    Xf = X[-1:, :] 
    X = X[7:, :]  # Excluding the first 7 rows for training

    # Model Building
    inputs = keras.Input(shape=(X.shape[1],))
    last_layer = inputs
    if params.get('batchnorm', False):
        last_layer = keras.layers.BatchNormalization()(inputs)
    if params.get('dropout', False):
        last_layer = keras.layers.Dropout(params['dropout_rate'])(last_layer)

    # First hidden layer with regularization
    hidden = keras.layers.Dense(params['neurons_1'],
                                activation=params['activation_1'],
                                kernel_regularizer=keras.regularizers.L1(
                                    params.get('h1_kernel_rate_l1', 0.0)),
                                activity_regularizer=keras.regularizers.L1(
                                    params.get('h1_activation_rate_l1', 0.0)))(last_layer)

    # Second hidden layer with regularization
    hidden = keras.layers.Dense(params['neurons_2'],
                                activation=params['activation_2'],
                                kernel_regularizer=keras.regularizers.L1(
                                    params.get('h2_kernel_rate_l1', 0.0)),
                                activity_regularizer=keras.regularizers.L1(
                                    params.get('h2_activation_rate_l1', 0.0)))(hidden)

    param_layers = []
    param_names = ["loc", "scale", "tailweight", "skewness", "power"]  # distribution type

    for p in range(paramcount[distribution]):
        regularize_param_kernel = params['regularize_' + param_names[p]]
        param_kernel_rate = (0.0 if not regularize_param_kernel
                             else params[str(param_names[p]) + '_rate_l1'])
        param_layers.append(keras.layers.Dense(
            168, activation='linear',  # kernel_initializer='ones',
            kernel_regularizer=keras.regularizers.L1(param_kernel_rate))(hidden))

    linear = tf.keras.layers.concatenate(param_layers)

    if distribution == 'Normal':
        outputs = tfp.layers.DistributionLambda(
            lambda t: tfd.Normal(loc=t[..., :168], scale=1e-3 + 3 * tf.math.softplus(t[..., 168:])))(linear)
    elif distribution == 'JSU':
        outputs = tfp.layers.DistributionLambda(
            lambda t: tfd.JohnsonSU(
                loc=t[..., :168],
                scale=1e-3 + 3 * tf.math.softplus(t[..., 168:336]),
                tailweight=1 + 3 * tf.math.softplus(t[..., 336:504]),
                skewness=t[..., 504:]))(linear)
    else:
        raise ValueError(f'Incorrect distribution {distribution}')

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(params['learning_rate']),
                  loss=lambda y, rv_y: -rv_y.log_prob(y),
                  metrics=['mae'])

    callbacks = [keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)]

    perm = np.random.permutation(np.arange(X.shape[0]))
    VAL_DATA = 0.2
    trainsubset = perm[:int((1 - VAL_DATA) * len(perm))]
    valsubset = perm[int((1 - VAL_DATA) * len(perm)):]

    model.fit(X[trainsubset], Y[trainsubset], epochs=100, validation_data=(X[valsubset], Y[valsubset]),
              callbacks=callbacks, batch_size=32, verbose=1)


    Y_real = Y[dayno:dayno+1, :]
    x_row = X[dayno:dayno+1, :]
    
    print(f"x_row shape: {x_row.shape}")
    dist = model(x_row)
    Y_pred = model.predict(x_row)
    
    columns = {
        'real': Y_real.flatten(),
        'predicted': Y_pred.flatten(),
        'loc': dist.loc.numpy().flatten(), 
        'scale': dist.scale.numpy().flatten(), 
    }
    if distribution == 'Normal':
       pass
    elif distribution == 'JSU':
        columns['tailweight'] = dist.tailweight.numpy().flatten()
        columns['skewness'] = dist.skewness.numpy().flatten()

    # Add forecast parameters
    # y_len = Y.shape[0]
    # df_idx_start = len(df) - (y_len - dayno) * 24
    df_forecast = pd.DataFrame(columns, index=df.index[0:168])
    # Save the DataFrame to CSV with correct date range
    filename = f'forecast_with_parameters_{dayno}.csv'
    df_forecast.to_csv(filename)
    print('saved to: ', filename)
    return df_forecast

In [ ]:
# point forecasting
best_params ={'dropout': True,
 'dropout_rate': 0.005579447300070095,
 'regularize_h1_activation': True,
 'regularize_h1_kernel': False,
 'h1_activation_rate_l1': 9.510278864916939e-05,
 'neurons_1': 75,
 'activation_1': 'elu',
 'regularize_h2_activation': False,
 'regularize_h2_kernel': True,
 'h2_kernel_rate_l1': 0.002614976865913897,
 'neurons_2': 55,
 'activation_2': 'relu',
 'regularize_loc': False,
 'regularize_scale': False,
 'regularize_tailweight': True,
 'tailweight_rate_l1': 0.0017295629298938247,
 'regularize_skewness': False,
 'learning_rate': 0.002625275649801325}
inputlist = [(best_params, day) for day in range(len(data) // 24)]
print(len(inputlist))

### normal distribution

In [ ]:
best_params ={
 'dropout': False,
 'regularize_h1_activation': True,
 'regularize_h1_kernel': False,
 'h1_activation_rate_l1': 1.3411937832055677e-05,
 'neurons_1': 71,
 'activation_1': 'relu',
 'regularize_h2_activation': False,
 'regularize_h2_kernel': True,
 'h2_kernel_rate_l1': 0.0004665633669008087,
 'neurons_2': 53,
 'activation_2': 'softplus',
 'regularize_loc': False,
 'regularize_scale': False,
 'learning_rate': 0.09157521767426445}
inputlist = [(best_params, day) for day in range(len(data) // 24)]
print(len(inputlist))

In [ ]:
# all_forecasts_NN = pd.DataFrame()


for e in inputlist:
    forecast= runoneday(e)
    # all_forecasts_NN = pd.concat([all_forecasts_NN, forecast])
# # Save all forecasts to a single CSV file
# all_forecasts_NN.to_csv('forecasts_NN_jsu2.csv')